<a href="https://colab.research.google.com/github/zelalemamera-stonybrook/projects-sandbox/blob/main/Recurrent_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
import torch
import torch.nn as nn
import torchaudio
import torchaudio.transforms as T
!pip install torchcodec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.8 MB/s eta 0:00:00


In [2]:
from torch.nn.utils.rnn import pack_sequence

In [26]:
a = torch.randint(0, 100, size=(5,))
b = torch.randint(0,100,size=(10,))
c = torch.randint(0,100, size=(3,))
d = torch.randint(0,100, size=(7,))
e = torch.randint(0, 100, (8,))
print(a)
print(b)
print(c)
print(d)
print(e)
packed = pack_sequence([a,b,c,d,e], enforce_sorted=False)

tensor([24, 80, 91, 43, 52])
tensor([26, 87, 96, 26,  9, 58, 71,  7, 15, 26])
tensor([98, 38,  4])
tensor([31,  8, 31, 54, 41, 12, 85])
tensor([27, 46, 12, 50, 54, 42, 16, 26])


In [27]:
packed

PackedSequence(data=tensor([26, 27, 31, 24, 98, 87, 46,  8, 80, 38, 96, 12, 31, 91,  4, 26, 50, 54,
        43,  9, 54, 41, 52, 58, 42, 12, 71, 16, 85,  7, 26, 15, 26]), batch_sizes=tensor([5, 5, 5, 4, 4, 3, 3, 2, 1, 1]), sorted_indices=tensor([1, 4, 3, 0, 2]), unsorted_indices=tensor([3, 0, 4, 2, 1]))

In [31]:
a = torch.rand((1,2,3))
b = torch.rand([1,2,4])
print(a, f"\n{b}")

tensor([[[0.0598, 0.4744, 0.1710],
         [0.7765, 0.6359, 0.1096]]]) 
tensor([[[0.9639, 0.4345, 0.5113, 0.4837],
         [0.5148, 0.5659, 0.7505, 0.9326]]])


In [110]:
a = torch.tensor([3.0, 2, 1], requires_grad=True)
b = torch.tensor([5.0, 6, 7], requires_grad=True)
print(a)
print(b)
c = (a ** 2) @ b
print(c)

tensor([3., 2., 1.], requires_grad=True)
tensor([5., 6., 7.], requires_grad=True)
tensor(76., grad_fn=<DotBackward0>)


In [94]:
dist = c ** 2
dist2 = 2 + dist
dist3 = dist2 ** 2

In [111]:
torch.autograd.backward(c)

In [87]:
print(dist3.grad, dist3.grad_fn.next_functions, dist2.grad_fn.next_functions, f"\n{dist.grad_fn.next_functions}", c.grad_fn.next_functions,
      f"\n{a.grad_fn}", b.grad_fn)

None ((<AddBackward0 object at 0x7c5975f11750>, 0),) ((<PowBackward0 object at 0x7c5975b1aa10>, 0), (None, 0)) 
((<DotBackward0 object at 0x7c5975f1ab30>, 0),) ((<AccumulateGrad object at 0x7c5975b45000>, 0), (<AccumulateGrad object at 0x7c5975b45360>, 0)) 
None None


/tmp/ipython-input-2088928086.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  print(dist3.grad, dist3.grad_fn.next_functions, dist2.grad_fn.next_functions, f"\n{dist.grad_fn.next_functions}", c.grad_fn.next_functions,


In [113]:
waveform, samplerate = torchaudio.load('/content/drive/MyDrive/Train_audio/audio1.wav')

In [114]:
class RNN(nn.RNNBase):

  def _init_(self, input_size, hidden_size, output_size):
    super().__init__()
    self.output_matrix = torch.rand((self.output_size, self.hidden_size))
    self.softmax = nn.Softmax(dim=0)
    self.output_size = output_size
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.hidden_layer = nn.RNNCell(input_size, hidden_size)

  def forward(self, input_sequence):
    hidden_list = []
    h0 = torch.zeros((self.hidden_size))
    for input in input_sequence:
      h0 = self.hidden_layer(input, h0)
      hidden_list.append(h0)
    output_hidden_layer = hidden_list[-1]
    output = self.softmax(self.output_matrix @ output_hidden_layer)
    return output


In [ ]:
def train(model, input_data, target_data):
  '''
  '''
